In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
from larndsim import consts, quenching, drifting, detsim, pixels_from_track
import importlib
importlib.reload(drifting)
importlib.reload(quenching)
importlib.reload(pixels_from_track)
importlib.reload(consts)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d.art3d as art3d

import pickle
import numpy as np
import numba as nb
import pandas as pd
import time

from numba import cuda
import cupy as cp
from math import ceil

Input Arrays

In [3]:
tracks_dict = pickle.load(open('edepsimDataset.p', 'rb'))

trackID_device = cp.array(tracks_dict['trackID'])
x_start_device = (cp.array(tracks_dict['x_start']))/1e2
y_start_device = (cp.array(tracks_dict['y_start']))/1e2
z_start_device = (cp.array(tracks_dict['z_start']))/1e2
x_end_device = (cp.array(tracks_dict['x_end']))/1e2
y_end_device = (cp.array(tracks_dict['y_end']))/1e2
z_end_device = (cp.array(tracks_dict['z_end']))/1e2

x_mid_device = (x_start_device + x_end_device)/2.
y_mid_device = (y_start_device + y_end_device)/2.
z_mid_device = (z_start_device + z_end_device)/2.

start_pos_device = cp.vstack([x_start_device, y_start_device, z_start_device]).T
end_pos_device = cp.vstack([x_end_device, y_end_device, z_end_device]).T

dE_device = cp.abs((cp.array(tracks_dict['dE'])))*1e3
dx_device = cp.sqrt(cp.sum((end_pos_device - start_pos_device)**2, axis=1))
dEdx_device = dE_device/dx_device

Declare Output Arrays

In [4]:
t_start_device = cp.zeros_like(x_start_device)
t_mid_device = cp.zeros_like(x_mid_device)
t_end_device = cp.zeros_like(x_end_device)

nElectrons_device = cp.zeros_like(x_start_device)
long_diffusion_device = cp.zeros_like(x_start_device)
tran_diffusion_device = cp.zeros_like(x_start_device)
arr_device = cp.zeros_like(x_start_device)

active_pixels_device = cp.zeros(shape=(x_start_device.shape[0], 10, 2))
neighboring_pixels_device = cp.zeros(shape =(x_start_device.shape[0], 100, 2))
n_pixels_device = cp.zeros(shape =(x_start_device.shape[0]))

GPU CONFIG

In [5]:
TPB = 128
BPG = ceil(x_start_device.shape[0] / TPB)

QUENCHING

In [6]:
quenching.Quench[TPB,BPG](dEdx_device, dE_device, nElectrons_device)

In [7]:
cuda.synchronize()

DRIFITING

In [8]:
drifting.Drift[TPB,BPG](z_start_device, z_end_device, z_mid_device,
                        t_start_device, t_end_device, t_mid_device,
                        nElectrons_device, long_diffusion_device, tran_diffusion_device)

In [9]:
cuda.synchronize()

IMPACTED PIXELS

In [10]:
pixels_from_track.get_pixels[TPB,BPG](x_start_device, y_start_device, x_end_device, y_end_device, 
                                      active_pixels_device, neighboring_pixels_device, n_pixels_device)

In [11]:
cuda.synchronize()

In [28]:
total_pixels = cp.sum(n_pixels_device)
trackID = cp.zeros(shape=(int(total_pixels)))
pixelID = cp.zeros(shape =(int(total_pixels),2))

In [24]:
for i in range(10):
    npix = int(n_pixels_device[i])
    stride = i * npix
    for trk in range(npix):
        trackID[trk + stride] = trackID_device[i]
        pixelID[trk + stride] = neighboring_pixels_device[i][trk]

In [25]:
trackID[0:50]

array([9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.,
       9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.,
       9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.])

In [27]:
pixelID[0:50]

array([[  2., 170.],
       [  2., 171.],
       [  2., 172.],
       [  2., 173.],
       [  2., 174.],
       [  3., 170.],
       [  3., 171.],
       [  3., 172.],
       [  3., 173.],
       [  3., 174.],
       [  4., 170.],
       [  4., 171.],
       [  4., 172.],
       [  4., 173.],
       [  4., 174.],
       [  5., 170.],
       [  5., 171.],
       [  5., 172.],
       [  5., 173.],
       [  5., 174.],
       [  6., 170.],
       [  6., 171.],
       [  6., 172.],
       [  6., 173.],
       [  6., 174.],
       [ -8., 176.],
       [ -8., 177.],
       [ -8., 178.],
       [ -8., 179.],
       [ -8., 180.],
       [ -7., 176.],
       [ -7., 177.],
       [ -7., 178.],
       [ -7., 179.],
       [ -7., 180.],
       [ -6., 176.],
       [ -6., 177.],
       [ -6., 178.],
       [ -6., 179.],
       [ -6., 180.],
       [ -5., 176.],
       [ -5., 177.],
       [ -5., 178.],
       [ -5., 179.],
       [ -5., 180.],
       [ -4., 176.],
       [ -4., 177.],
       [ -4.,